In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np
import pickle

os.chdir('../..')
os.getcwd()

#from graph_creator.MapGraph import MapGraph
#from graph_creator.ActorGraph import ActorGraph

from graph_creator.plot_graphs import plot_lane_map_advanced, add_actors_to_map, add_actor_edges_to_map

### Start comparing graphs:

In [3]:
from glob import glob
import pickle
import networkx as nx
import random
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

graph_paths = glob("/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/*.pkl")
len(graph_paths)

7175

In [4]:
random.sample(list(range(len(graph_paths))), 10)

[633, 5586, 6930, 4054, 3373, 1932, 3324, 6735, 1503, 6921]

In [4]:
# Define a random set of coverage situations with 3 nodes each
# This still needs to be replaced by more strategically selected graphs

#idxs = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
idxs = random.sample(list(range(len(graph_paths))), 10)

coverage_graphs = {}
cov_data = {}

for idx in idxs:
    graph_path = graph_paths[0]
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)
    #random.sample(ag_nx.nodes, 3)
    coverage_graphs[idx] = ag_nx.subgraph(random.sample(list(ag_nx.nodes), 3))
    cov_data[idx] = []

cov_data["degree"] = []
cov_data["density"] = []
cov_data["diameter"] = []


In [5]:
ag_nx.nodes(data=True)

NodeDataView({'594': {'lane_id': '6_1', 'lane_ids': ['6_1'], 's': 24.0, 'xyz': <POINT Z (301.34 327.05 0.246)>, 'lon_speed': 0.0, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>}, '581': {'lane_id': '19_1', 'lane_ids': ['19_1'], 's': 67.74999283254147, 'xyz': <POINT Z (338.98 249.43 0.246)>, 'lon_speed': -0.0, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>}, '589': {'lane_id': '19_-1', 'lane_ids': ['19_-1'], 's': 89.24998013675213, 'xyz': <POINT Z (335.49 298.81 0.246)>, 'lon_speed': 0.0, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>}, '593': {'lane_id': '12_1', 'lane_ids': ['12_1'], 's': 130.25, 'xyz': <POINT Z (195.005 195.27 0.246)>, 'lon_speed': 0.0, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>}, '597': {'lane_id': '12_-1', 'lane_ids': ['12_-1'], 's': 40.5, 'xyz': <POINT Z (142.361 198.76 0.246)>, 'lon_speed': 0.0, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>}, '570': {'lane_id': '12_1', 'lane_ids': ['12_1'], 's': 112.74998474121094, 'xyz': <POINT Z (212.642 194.773 0.246)>, 'lon_speed':

In [8]:
for graph_path in graph_paths:
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)

    for key in coverage_graphs.keys():
        GM = nx.algorithms.isomorphism.DiGraphMatcher(
            ag_nx, coverage_graphs[key],
            node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
            # node_match=nx.algorithms.isomorphism.categorical_node_match(['actor_type', 'lane_change'], [None, None]),
            edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
        )
        cov_data[key].append(GM.subgraph_is_isomorphic())
    
    cov_data["degree"].append(sum(dict(ag_nx.degree()).values()) / len(ag_nx.nodes()))
    cov_data["density"].append(nx.density(ag_nx))
    cov_data["diameter"].append(nx.diameter(ag_nx))

cov_data_df = pd.DataFrame(cov_data)
cov_data_df.mean()

416         0.307456
647         0.546063
2398        0.307456
4275        0.307456
4983        0.307456
5349        0.307456
5314        0.307456
3032        0.354983
3620        0.354983
111         0.307456
degree      2.705190
density     0.669820
diameter    3.215889
dtype: float64

In [ ]:
list(cov_data_df.columns[:10])


In [ ]:
cov_data_df.groupby(list(cov_data_df.columns[:11]), as_index=False).size()

In [ ]:
# sns.pairplot(data=cov_data_df)
# plt.show()


# Notebook junkyard